In [10]:
import os
import tempfile
from pdf2image import convert_from_path, convert_from_bytes
 
images = convert_from_bytes(open('/home/ajdinb/Desktop/pdfs/Sales/ASK Dubai - Fundraiser/JSTI_Alex Wu_VP.pdf', 'rb').read())

In [13]:
i = 0
for page in images:
    page.save('output/out'+str(i)+'.jpg', 'JPEG')
    i+=1